CS524: Introduction to Optimization Lecture 14
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 4, 2024
--------------

#  Picnic problem

The Hatfields, Montagues, McCoys and Capulets are going on their annual
family picnic.  Four cars are available to transport the families to
the picnic.  The cars can carry the following numbers of people:
car 1, 4; car 2, 3; car 3, 3; car 4, 4.
There are four people in each family, and no car can carry more than two
people from any one family.  Determine the maximum number of people that 
can be transported to the picnic.

In [3]:
import sys
import pandas as pd
import numpy as np

from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
    ModelStatus,SolveStatus,
)

options = Options(equation_listing_limit=8)
m = Container(options=options)

In [4]:
# Order is important here since later use i.first and i.last
i = Set(m,'i',description='all nodes of problem',
    records=['s']+['hatfield','montague','mccoy','capulet']+[f'car{ind+1}' for ind in range(4)]+['t'])

# Relevant subsets for picnic problem
family = Set(m,'family',domain=i,records=['hatfield','montague','mccoy','capulet'])
car = Set(m,'car',domain=i,records=[f"car{ind+1}" for ind in range(4)])

cap = Parameter(m,'cap',domain=car, records=[('car1', 4), ('car2', 3), ('car3', 3), ('car4', 4)])
famSize = Parameter(m,'famSize',domain=family)
famSize[family] = 4
perFam = Parameter(m,'perFam',domain=family)
perFam[family] = 2

num = Variable(m,'num','positive',domain=[family,car],description='Number of people from family in car')
car_sink = Variable(m,'car_sink','positive',domain=car,description='Number of people from car to sink')
source_fam = Variable(m,'source_fam','positive',domain=family,description='Flow from source to family node')
totgo = m.addVariable('totgo','free',description='total people going to picnic')

car_bal = Equation(m,'car_bal',domain=car)
car_bal[car]= Sum(family, num[family,car]) == car_sink[car]

family_bal = Equation(m,'family_bal',domain=family)
family_bal[family]= source_fam[family] == Sum(car, num[family,car])

In [5]:
maxflow = Model(m,'maxflow',equations=m.getEquations(),sense=Sense.MAX,problem=Problem.LP,
    objective=Sum(family, source_fam[family]))

source_fam.up[family] = famSize[family]
num.up[family,car] = perFam[family]
car_sink.up[car] =  cap[car]

resusd = {}
# use network simplex method (in option file)
maxflow.solve(solver='cplex',output=None)
resusd['lp'] = maxflow.total_solve_time

# Reset initial values to test speed
source_fam.l[family] = 0
car_sink.l[car] = 0
num.l[family,car] = 0
# use network simplex method (in option file)
maxflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)
resusd['net'] = maxflow.total_solve_time

# POST PROCESSING #

print(f"Objective Function Value: {round(maxflow.objective_value,4)}\n")
print("num travel:"); display(num.pivot())
print("resusd:\n", resusd)


Objective Function Value: 14.0

num travel:


,car1,car2,car3,car4
hatfield,2.0,1.0,0.0,1.0
montague,0.0,0.0,2.0,2.0
mccoy,2.0,2.0,0.0,0.0
capulet,0.0,0.0,1.0,1.0


resusd:
 {'lp': 0.06700011435896158, 'net': 0.026000034995377064}


In [6]:
# Redo with nodes and arcs
j = m.addAlias('j',i)
k = m.addAlias('k',i)
s = m.addSet('s',domain=i,description='sources',records=['s'])
t = m.addSet('t',domain=i,description='sinks',records=['t'])

arcs = m.addSet('arcs',domain=[i,j])
arcs[s,family] = True
arcs[family,car] = True
arcs[car,t] = True

u = m.addParameter('u',domain=[i,j])
u[s,family] = famSize[family]
u[family,car] = perFam[family]
u[car,t] = cap[car]

# VARIABLES #

x = Variable(m,"x","positive",domain=[i,j],description="Number of people from family in car")

# EQUATIONS #

balance = m.addEquation('balance',domain=[i])
balance[i].where[~s[i] & ~t[i]]= Sum(arcs[i,k], x[i,k]) - Sum(arcs[j,i], x[j,i]) == 0

maxflow = Model(m,
    name="maxflow",
    equations=[balance],
    problem=Problem.LP,
    sense=Sense.MAX,
    objective=Sum(arcs[s,j], x[s,j]),
)

x.up[arcs] = u[arcs]

maxflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

ImplicitParameter(parent=Parameter(name=perFam, domain=[Set(name=family, domain=[Set(name=i, domain=['*'])])]), name=perFam, domain=[Set(name=family, domain=[Set(name=i, domain=['*'])])], permutation=None), parent_scalar_domains=[])


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,14,9,25,LP,CPLEX,0.005


# Dual Model

In [5]:
# DUAL MODEL

pi = m.addVariable("pi","positive",domain=[i,j])
phi = m.addVariable("phi","free",domain=[i])

dualcons = m.addEquation('dualcons',domain=[i,j])
dualcons[i,j].where[arcs[i,j]]= phi[i].where[~s[i]] + pi[i,j] - phi[j].where[~t[j]] >= Number(1).where[s[i]]

dualflow = m.addModel(
    name="dualflow",
    equations=[dualcons],
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(arcs, u[arcs]*pi[arcs]),
)

dualflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

# POST PROCESSING #

print(f"Objective Function Value: {round(dualflow.objective_value, 4)}\n")
# display(pi.records,phi.records)
print("pi:\n", pi.records[["i","j","level"]])
print("phi:\n", phi.records[["i","level"]])


Objective Function Value: 14.0

pi:
            i         j  level
0          s  hatfield    0.0
1          s  montague    0.0
2          s     mccoy    0.0
3          s   capulet    0.0
4   hatfield      car1    0.0
5   hatfield      car2    0.0
6   hatfield      car3    0.0
7   hatfield      car4    0.0
8   montague      car1    0.0
9   montague      car2    0.0
10  montague      car3    0.0
11  montague      car4    0.0
12     mccoy      car1    0.0
13     mccoy      car2    0.0
14     mccoy      car3    0.0
15     mccoy      car4    0.0
16   capulet      car1    0.0
17   capulet      car2    0.0
18   capulet      car3    0.0
19   capulet      car4    0.0
20      car1         t    1.0
21      car2         t    1.0
22      car3         t    1.0
23      car4         t    1.0
phi:
           i  level
0  hatfield   -1.0
1  montague   -1.0
2     mccoy   -1.0
3   capulet   -1.0
4      car1   -1.0
5      car2   -1.0
6      car3   -1.0
7      car4   -1.0


# Path form of problem

In [6]:
# Each path is indexed by a (family,car) pair
# a path = source -> family -> car -> sink

p = Set(m,'p', domain=[i,j])
p[family,car] = True

# PATH FLOW MODEL #

f = Variable(m,"f","positive",domain=[i,j],description=" Flow on unique path through family and car") 
 
linkb = Equation(m,'linkb',domain=[i,j])
linkb[i,j].where[arcs[i,j]]= (
    # path flow on arc from family i to car j
    f[i,j].where[p[i,j]]
    # all paths going through family j
    + Sum(k.where[arcs[j,k] & i.first], f[j,k])
    # all paths using car i
    + Sum(k.where[arcs[k,i] & j.last], f[k,i])
    <= u[i,j] )

pflow = Model(m,
    name="pflow",
    equations=[linkb],
    problem=Problem.LP,
    sense=Sense.MAX,
    objective=Sum(p, f[p]),
)

pflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

# POST PROCESSING #

print(f"Objective Function Value: {round(pflow.objective_value, 4)}\n")
print(f"Number of equations: {pflow.num_equations:.0f}\n")
print("f:") 
display(f.records[['i','j','level']])  

Objective Function Value: 14.0

Number of equations: 25

f:


,i,j,level
0,hatfield,car1,0.0
1,hatfield,car2,1.0
2,hatfield,car3,0.0
3,hatfield,car4,2.0
4,montague,car1,2.0
5,montague,car2,0.0
6,montague,car3,1.0
7,montague,car4,0.0
8,mccoy,car1,2.0
9,mccoy,car2,0.0


# Dual path model for picnic problem

In [7]:
# DUAL MODEL

dualcons[arcs[i,j]]= phi[i].where[~s[i]] + pi[i,j] - phi[j].where[~t[j]] >= Number(1).where[s[i]]
dualcons[p[i,j]]= pi['s',i] + pi[i,j] + pi[j,'t'] >= 1

dualflow = Model(m,
    name="dualflow",
    equations=[dualcons],
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(arcs, u[arcs]*pi[arcs]),
)

dualflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

# POST PROCESSING #

print(f"Objective Function Value: {round(dualflow.objective_value, 4)}\n")
print("pi:") 
display(pi.records[["i","j","level"]])
# but these are just marginals on linkb
print("linkb marginals:"); display(linkb.records[['i','j','marginal']])
# Note that marginal on dualcons are also a solution to primal (but soln not necc. unique)
print("dualcons marginals:"); display(dualcons.records)

Objective Function Value: 14.0

pi:


,i,j,level
0,s,hatfield,0.0
1,s,montague,0.0
2,s,mccoy,0.0
3,s,capulet,0.0
4,hatfield,car1,0.0
5,hatfield,car2,0.0
6,hatfield,car3,0.0
7,hatfield,car4,0.0
8,montague,car1,0.0
9,montague,car2,0.0


linkb marginals:


,i,j,marginal
0,s,hatfield,0.0
1,s,montague,0.0
2,s,mccoy,-0.0
3,s,capulet,-0.0
4,hatfield,car1,0.0
5,hatfield,car2,0.0
6,hatfield,car3,0.0
7,hatfield,car4,-0.0
8,montague,car1,0.0
9,montague,car2,0.0


dualcons marginals:


,i,j,level,marginal,lower,upper,scale
0,s,hatfield,1.0,4.0,1.0,inf,1.0
1,s,montague,1.0,4.0,1.0,inf,1.0
2,s,mccoy,1.0,4.0,1.0,inf,1.0
3,s,capulet,1.0,2.0,1.0,inf,1.0
4,hatfield,car1,1.0,2.0,1.0,inf,1.0
5,hatfield,car2,1.0,2.0,1.0,inf,1.0
6,hatfield,car3,1.0,0.0,1.0,inf,1.0
7,hatfield,car4,1.0,0.0,1.0,inf,1.0
8,montague,car1,1.0,2.0,1.0,inf,1.0
9,montague,car2,1.0,1.0,1.0,inf,1.0
